## WFP-01-02-02 Landsat-8 reflectances and vegetation indices

As a land cover change analyst I want to derive the vegetation indices (NDVI, NDWI, MNDWI, NDBI) from Landsat-8 products.

### <a name="service">Service Definition

In [1]:
service = dict([('title', 'WFP-01-02-02 Landsat-8 reflectances and vegetation indices'),
                ('abstract', 'WFP-01-02-02 Landsat-8 reflectances and vegetation indices'),
                ('id', 'ewf-wfp-01-02-02')])

### <a name="parameter">Parameter Definition 

**None**


### <a name="runtime">Runtime parameter definition

**Input identifier**

This is the Landsat-8 product identifier

In [5]:
input_identifier = 'LC81540332017296LGN00'

**Input reference**

This is the Landsat-8 catalogue reference

In [6]:
input_reference = 'https://catalog.terradue.com/landsat8/search?format=atom&uid=LC81540332017296LGN00'

**Data path**

This path defines where the data is staged-in. 

In [7]:
data_path = '/workspace/data'

### <a name="workflow">Workflow

#### Import the packages required for processing the Landsat-8 vegetation indices

In [8]:
import os
import sys
import glob

sys.path.append('/opt/OTB-6.2.0/lib/python')
sys.path.append('/opt/OTB-6.2.0/lib/libfftw3.so.3')
os.environ['OTB_APPLICATION_PATH'] = '/opt/OTB-6.2.0/lib/otb/applications'
os.environ['LD_LIBRARY_PATH'] = '/opt/OTB-6.2.0/lib'
os.environ['ITK_AUTOLOAD_PATH'] = '/opt/OTB-6.2.0/lib/otb/applications'
os.environ['GDAL_DATA'] = '/opt/anaconda/share/gdal/'
import otbApplication

In [9]:
def get_attribute(mtl_path, attribute):
    
    mtl = open(mtl_path, 'r')
    attribs = mtl.readlines()
        
    for line in attribs:
        
        if attribute + ' = ' in line:
            return line.split('=')[1].rstrip().strip(' ').strip('""')
      

In [10]:
def offset(band):
    
    return get_attribute(ls8_metadata, 'REFLECTANCE_ADD_BAND_' + str(band))

In [11]:
def gain(band):
    
    return get_attribute(ls8_metadata, 'REFLECTANCE_MULT_BAND_' + str(band))

In [12]:
def dn_to_reflectance(band):
    
    ls8_gain = gain(band)
    ls8_offset = offset(band)
    ls8_band_filename = get_attribute(ls8_metadata, 'FILE_NAME_BAND_' + str(band))
    
    otb_app = otbApplication.Registry.CreateApplication('BandMath')

    otb_app.SetParameterStringList('il', [ls8_band_filename])

    otb_app.SetParameterString('out', '%s_R%s.TIF' % (get_attribute(ls8_metadata, 'LANDSAT_PRODUCT_ID'),
                                                     str(band)))
    
    otb_app.SetParameterString('exp', '%s * im1b1 + %s' % (ls8_gain, ls8_offset))

    otb_app.ExecuteAndWriteOutput()
      
    return True
    

In [13]:
def normalized_difference(band_1, band_2, suffix = ''):
    
    otb_app = otbApplication.Registry.CreateApplication('BandMath')

    otb_app.SetParameterStringList('il', [band_1, band_2])

    otb_app.SetParameterString('out', '%s_%s.TIF' % (get_attribute(ls8_metadata, 'LANDSAT_PRODUCT_ID'),
                                                     suffix))
    
    otb_app.SetParameterString('exp', 'im1b1 >= 0 && im1b1 <= 1 && im2b1 >= 0 && im2b1 <= 1 ? ( im1b1 - im2b1 ) / ( im1b1 + im2b1 ) : 0 ')

    otb_app.ExecuteAndWriteOutput()
      
    return True
    

In [14]:
import tarfile

ls8_tar = os.path.join(data_path, input_identifier + '.tar')

tar = tarfile.open(ls8_tar)
tar.extractall()
tar.close()

In [15]:
search_expression = ('*_MTL.txt')

In [16]:
ls8_metadata = glob.glob(search_expression)[0]

In [17]:
ls8_metadata

'LC08_L1TP_154033_20171023_20171107_01_T1_MTL.txt'

#### Process the reflectances

In [18]:
for band in range(1,10):
    print band
    dn_to_reflectance(band)




1
2
3
4
5
6
7
8
9


#### Process the vegetation indexes

In [19]:
for indice in ['NDVI', 'NDWI', 'MNDWI', 'NDBI']:
    
    if indice == 'NDVI':
        band_1 = 4
        band_2 = 5
    
    if indice == 'NDWI':
        band_1 = 3
        band_2 = 5
    
    if indice == 'MNDWI':
        band_1 = 3
        band_2 = 6
        
    if indice == 'NDBI':
        band_1 = 5
        band_2 = 4   
    
        
    normalized_difference('%s_R%s.TIF' % (get_attribute(ls8_metadata, 'LANDSAT_PRODUCT_ID'), str(band_1)),
                          '%s_R%s.TIF' % (get_attribute(ls8_metadata, 'LANDSAT_PRODUCT_ID'), str(band_2)),
                          indice)
    

#### Clean-up

In [20]:
for band in range(1,12):
    
    os.remove(get_attribute(ls8_metadata, 'FILE_NAME_BAND_' + str(band)))
    
for ls8_file in [get_attribute(ls8_metadata, 'FILE_NAME_BAND_QUALITY'),
                 get_attribute(ls8_metadata, 'ANGLE_COEFFICIENT_FILE_NAME'),
                 get_attribute(ls8_metadata, 'METADATA_FILE_NAME')]:
    
    os.remove(ls8_file)

#### Metadata

### <a name="license">License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.